In [1]:
import nd2
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [13]:
# load data:
data_folder = r'/Users/puzheng/Documents/MERFISH/DY'
for _fl in os.listdir(data_folder):
    if _fl.split(os.extsep)[-1] == 'nd2':
        print(_fl)
        with nd2.ND2File(os.path.join(data_folder, _fl)) as ndfile:
            print(ndfile.metadata)
            metadata = ndfile.metadata
            im_stack = ndfile.asarray()
            voxel_sizes = ndfile.voxel_size()
    break

20240814_232340_437__Channel60x CY5,60x DSRED,60x GFP_Seq0008 - Denoised.nd2
Metadata(contents=Contents(channelCount=3, frameCount=312), channels=[Channel(channel=ChannelMeta(name='60x CY5', index=0, color=Color(r=255, g=0, b=255, a=1.0), emissionLambdaNm=700.0, excitationLambdaNm=None), loops=LoopIndices(NETimeLoop=None, TimeLoop=None, XYPosLoop=0, ZStackLoop=1), microscope=Microscope(objectiveMagnification=60.0, objectiveName='PLAN APO λD 60x OIL OFN25 DIC N2', objectiveNumericalAperture=1.42, zoomMagnification=1.0, immersionRefractiveIndex=1.515, projectiveMagnification=None, pinholeDiameterUm=50.0, modalityFlags=['fluorescence', 'spinningDiskConfocal']), volume=Volume(axesCalibrated=(True, True, True), axesCalibration=(0.1107206280606, 0.1107206280606, 0.5), axesInterpretation=('distance', 'distance', 'distance'), bitsPerComponentInMemory=16, bitsPerComponentSignificant=16, cameraTransformationMatrix=(-0.9999999001902034, 0.00044678807428339995, -0.00044678807428339995, -0.99999990

In [3]:
# output
output_folder = r'/Users/puzheng/Documents/MERFISH/MERFISH_Data/20240914-MF4'
if not os.path.exists(output_folder):
    print(f"Create output folder: {output_folder}")
    os.makedirs(output_folder)

In [14]:
position_df = {'fov':[], 'X':[], 'Y':[], 'Z':[]}
with nd2.ND2File(os.path.join(data_folder, _fl)) as ndfile:
    for _k, _v in ndfile.unstructured_metadata()['ImageMetadataLV']['SLxExperiment']['LoopPars']['Points'].items():
        #print(_k, _v['PosX'], _v['PosY'], _v['PosZ'])
        position_df['fov'].append(int(_k.split('i')[-1]))
        position_df['X'].append(_v['PosX'])
        position_df['Y'].append(_v['PosY'])
        position_df['Z'].append(_v['PosZ'])
    
    #print(ndfile.attributes)
position_df = pd.DataFrame(position_df).set_index('fov')
# save df
position_df.to_csv(os.path.join(output_folder, 'positions.csv'))
# save x-y position file:
position_df[['X','Y']].to_csv(os.path.join(output_folder, f"{20240914}_positions.txt"), float_format='%.2f', index=False)

In [ ]:
# save iamges:
# load data:
data_folder = r'/Users/puzheng/Documents/MERFISH/DY'
for _fl in os.listdir(data_folder):
    if _fl.split(os.extsep)[-1] == 'nd2':
        print(_fl)
        with nd2.ND2File(os.path.join(data_folder, _fl)) as ndfile:
            print(ndfile.metadata)
            metadata = ndfile.metadata
            im_stack = ndfile.asarray()
    break

In [18]:
for _im in im_stack:
    print(_im.shape)
    break

(13, 3, 2304, 2304)
